In [174]:
'''
Author: bob
Date: 2022-05-24 16:59:52
LastEditors: bob
LastEditTime: 2022-05-24 16:59:58
FilePath: \任务22-开发网络版对账单转换工具\main.ipynb
Description: 使用python编写的网络版对账单转换工具

Copyright (c) 2022 by bob, All Rights Reserved. 
'''


'\nAuthor: bob\nDate: 2022-05-24 16:59:52\nLastEditors: bob\nLastEditTime: 2022-05-24 16:59:58\nFilePath: \\任务22-开发网络版对账单转换工具\\main.ipynb\nDescription: 使用python编写的网络版对账单转换工具\n\nCopyright (c) 2022 by bob, All Rights Reserved. \n'

In [175]:
# open读取txt文件

with open('data.txt', 'r',encoding='utf-8') as f:
    data = f.read()
data

'212513 WMW51DK09N2R49823 X8CK7F\n5122073700 51227497233 89.38 100.00 21.00- 1 70.61 5.08 9.84 85.53\n5122073700 5199000 31.90 100.00 4 127.60 16.59 144.19\n_____________________________________________________________________________________________________________________________________\n* 赔付70.61 127.60 5.08 26.43 229.72\n212511 WBACY2105M9H82023 X8CK7G\n0061740600 0072693 31.90 100.00 8 255.20 33.18 288.38\n_____________________________________________________________________________________________________________________________________\n* 赔付255.20 33.18 288.38\n212510 WMW41DL05N3N29495 X8CK7H\n6411721200 0000556 31.90 100.00 2 63.80 8.29 72.09\n6411721200 6411841 31.90 100.00 4 127.60 16.59 144.19\n6411721200 64119384023 778.76 100.00 29.00- 1 552.92 39.81 77.05 669.78\n_____________________________________________________________________________________________________________________________________\n* 赔付552.92 191.40 39.81 101.93 886.06\n212499 WMWXR3108M2P29472 X8CK7I\n3234

In [176]:
# 1.通过re模块提取数据-表头
import re
p=r'(\d{5,6}) (\S{17}) (\S{6})'
res_head=re.findall(p,data)
res_head


[('212513', 'WMW51DK09N2R49823', 'X8CK7F'),
 ('212511', 'WBACY2105M9H82023', 'X8CK7G'),
 ('212510', 'WMW41DL05N3N29495', 'X8CK7H'),
 ('212499', 'WMWXR3108M2P29472', 'X8CK7I'),
 ('212517', 'WBA8X310XHG775436', 'X8CLBN'),
 ('212516', 'WBA7E4109KGV47166', 'X8CLBO'),
 ('212519', 'WMWXR3105L2L40383', 'X8CLUG'),
 ('212520', 'WMW51DH01N2R69751', 'X8CLUH'),
 ('212522', 'WMWYS5107K3F27527', 'X8CLUI'),
 ('212524', 'WMWLN7107K2J10358', 'X8CLUJ'),
 ('212530', 'WBA5A3104ED743163', 'X8CLUK'),
 ('212531', 'WMWXR1107KTE62372', 'X8CLUL'),
 ('212532', 'WMWXN7106G3A37196', 'X8CLUM'),
 ('435537', 'WBAWX3109E0G35537', 'X8CMG1'),
 ('802839', 'WBAKR0101G0U02839', 'X8CMG2'),
 ('612318', 'WBALS2100H0V12318', 'X8CMG3'),
 ('316705', 'MMFLS4405HY916705', 'X8CMG4'),
 ('244828', 'WBAVL3105EVS44828', 'X8CMG5'),
 ('386111', 'WMWXM7109F2A86111', 'X8CMG6'),
 ('254680', 'WBALS210XG0S54680', 'X8CMG7')]

In [177]:
# 2.通过re模块提取数据-表体
# import re
p=r'\*( 赔付)(\S*) ?(\S*) ?(\S*) ?(\S*) ?(\S*) ?(\S*) ?(\S*)'
res_body=re.findall(p,data)
res_body

[(' 赔付', '70.61', '127.60', '5.08', '26.43', '229.72', '', ''),
 (' 赔付', '255.20', '33.18', '288.38', '', '', '', ''),
 (' 赔付', '552.92', '191.40', '39.81', '101.93', '886.06', '', ''),
 (' 赔付', '4,238.14', '127.60', '305.15', '607.22', '5,278.11', '', ''),
 (' 赔付', '10,376.99', '574.20', '747.14', '1,520.79', '13,219.12', '', ''),
 (' 赔付', '3,278.85', '65.58', '434.78', '3,779.21', '', '', ''),
 (' 赔付', '145.14', '130.96', '431.57', '92.00', '799.67', '', ''),
 (' 赔付', '145.14', '130.96', '550.47', '107.46', '934.03', '', ''),
 (' 赔付', '145.14', '130.96', '382.12', '85.58', '743.80', '', ''),
 (' 赔付', '145.14', '130.96', '472.04', '97.27', '845.41', '', ''),
 (' 赔付', '224.98', '1,786.40', '16.19', '263.58', '2,291.15', '', ''),
 (' 赔付', '5,427.38', '478.50', '949.79', '891.23', '7,746.90', '', ''),
 (' 赔付', '3,774.95', '191.40', '660.62', '601.51', '5,228.48', '', ''),
 (' 赔付', '719.00', '93.47', '812.47', '', '', '', ''),
 (' 赔付', '830.84', '108.01', '938.85', '', '', '', ''),
 (' 赔付

In [178]:
# 3.通过re模块提取数据-对账单序列号
# import re
p=r'Credit Note Number (\d{7})'
res_number=re.findall(p,data)
res_number=res_number[0]
res_number

'6530279'

In [179]:
# 4.通过re模块提取数据-对账单日期
# import re
p=r'created on (\S{10})'
res_date=re.findall(p,data)
res_date=res_date[0]
res_date

'2022.08.08'

In [180]:

# 5.判断国产还是进口
first_char=res_head[0][1][0]
if first_char=='L':
    s='BBA'
else:
    s='GIS'
s

'GIS'

In [181]:
# 通过pandas转换成dataframe
import pandas as pd
import numpy as np
df1=pd.DataFrame(res_head,columns=['保修单号','车架号','DWP保修单号'])
df1
df2=pd.DataFrame(res_body,columns=list('A1234567'))
df2
# 合并两个dataframe concat axis=1
df=pd.concat([df1,df2],axis=1)
df

# 去掉',',并转换成float
df.loc[:,'1':'7']=df.loc[:,'1':'7'].applymap(lambda x:str(x).replace(',',''))

# 转换成float
for i in range(1,8):
    df.loc[:,'{}'.format(i)]=pd.to_numeric(df.loc[:,'{}'.format(i)],errors='coerce')

# 添加汇总列
df['总计']=df.loc[:,'1':'7'].max(axis=1)

# 添加各种'对账单序列号','对账单日期'列
df['对账单序列号']=res_number
df['对账单日期']=res_date.replace('.','/')

df.head()
# df.info()

,保修单号,车架号,DWP保修单号,A,1,2,3,4,5,6,7,总计,对账单序列号,对账单日期
0,212513,WMW51DK09N2R49823,X8CK7F,赔付,70.61,127.60,5.08,26.43,229.72,NaN,NaN,229.72,6530279,2022/08/08
1,212511,WBACY2105M9H82023,X8CK7G,赔付,255.20,33.18,288.38,NaN,NaN,NaN,NaN,288.38,6530279,2022/08/08
2,212510,WMW41DL05N3N29495,X8CK7H,赔付,552.92,191.40,39.81,101.93,886.06,NaN,NaN,886.06,6530279,2022/08/08
3,212499,WMWXR3108M2P29472,X8CK7I,赔付,4238.14,127.60,305.15,607.22,5278.11,NaN,NaN,5278.11,6530279,2022/08/08
4,212517,WBA8X310XHG775436,X8CLBN,赔付,10376.99,574.20,747.14,1520.79,13219.12,NaN,NaN,13219.12,6530279,2022/08/08



# 附加功能-找到每个细节

In [182]:
df.shape

(20, 14)

In [183]:
# 0.re提取每个特征-生成段落
import re
detail_list=re.findall(r'[^_]+',data)[:-1]
detail_list.__len__()
# detail_list[0]


20

In [184]:
# 1提取工时行
FRU=re.findall(r'(\S{10}) (\d{7}) (\S+) (\S+) (\S+) (\S+) (\S+) (\S+)',detail_list[0])
FRU
df_FRU=pd.DataFrame(FRU,columns=list('ABCDFHKL'))
df_FRU

,A,B,C,D,F,H,K,L
0,5122073700,5199000,31.9,100.0,4,127.6,16.59,144.19


In [185]:
# 2.提取零件行-正常配件
part_normal=re.findall(r'(\S{10}) (\S{11}) (\S+) (\S+) (\S+) (\S+) (\S+) (\S+) (\S+) (\S+)',detail_list[1])
part_normal
df_part_normal=pd.DataFrame(part_normal,columns=list('ABCDEFGJKL'))
df_part_normal

,A,B,C,D,E,F,G,J,K,L


In [186]:
# 3.提取零件行-BSI配件
part_BSI=re.findall(r'(8\S{8}6) (\S{11}) (\S+) (\S+) (\S+) (\S+) (\S+) (\S+)',detail_list[0])
part_BSI
df_part_BSI=pd.DataFrame(part_BSI,columns=list('ABCDFGKL'))
df_part_BSI

,A,B,C,D,F,G,K,L


In [187]:
# 4.提取辅料
sub_4=re.findall(r'(\S{10}) (\S{1}) (\S+) (\S+) (\S+) (\S+) (\S+) (\S+)',detail_list[0])
sub_4
df_sub_4=pd.DataFrame(sub_4,columns=list('ABCDFIKL'))
df_sub_4

,A,B,C,D,F,I,K,L


In [188]:
# 5.提取F111
F111=re.findall(r'(9999001386) (4) (\S+) (\S+) (\S+) (\S+) (\S+)',i)
F111
df_F111=pd.DataFrame(F111,columns=list('ABCFIKL'))
df_F111

,A,B,C,F,I,K,L
0,9999001386,4,713.0,1,713.0,92.69,805.69


In [189]:
# 5.找到单行的表头
p=r'(\d{5,6}) (\S{17}) (\S{6})'
one_head=re.findall(p,detail_list[0])
one_head
claim,vin,dwp=one_head[0]
claim,vin,dwp

('212513', 'WMW51DK09N2R49823', 'X8CK7F')

In [190]:
# 建立一个空的dataframe，只包括表头-作用是避免漏掉没有数据的列
df_na=pd.DataFrame(columns=list('ABCDEFGHIJKL'))
df_na

,A,B,C,D,E,F,G,H,I,J,K,L


In [191]:
# 拼接dataframe
res_one_part=pd.concat([df_FRU,df_part_normal,df_part_BSI,df_sub_4,df_F111],axis=0)
# column 按照字母排序
res_one_part.sort_index(axis=1,inplace=True)
# claim vin dwp 插入前三列
res_one_part.insert(0,'claim',claim)
res_one_part.insert(1,'vin',vin)
res_one_part.insert(2,'dwp',dwp)
res_one_part

,claim,vin,dwp,A,B,C,D,E,F,G,H,I,J,K,L
0,212513,WMW51DK09N2R49823,X8CK7F,5122073700,5199000,31.9,100.0,NaN,4,NaN,127.6,NaN,NaN,16.59,144.19
0,212513,WMW51DK09N2R49823,X8CK7F,9999001386,4,713.0,NaN,NaN,1,NaN,NaN,713.0,NaN,92.69,805.69


In [192]:
# 0.re提取每个特征-生成段落
import re
detail_list=re.findall(r'[^_]+',data)[:-1]
detail_list.__len__()

res_list=[]
for i in detail_list:
    
    concat_list=[]
    
    # 
    # 1提取工时行
    FRU=re.findall(r'(\S{10}) (\d{7}) (\S+) (\S+) (\S+) (\S+) (\S+) (\S+)',i)
    if FRU:
        df_FRU=pd.DataFrame(FRU,columns=list('ABCDFHKL'))
        concat_list.append(df_FRU)
    # df_FRU=pd.DataFrame(FRU,columns=list('ABCDFHKL'))
    # df_FRU
    

    # 
    #   2.提取零件行-正常配件
    part_normal=re.findall(r'(\S{10}) (\S{11}) (\S+) (\S+) (\S+) (\S+) (\S+) (\S+) (\S+) (\S+)',i)
    part_normal
    if part_normal:
        df_part_normal=pd.DataFrame(part_normal,columns=list('ABCDEFGJKL'))
        concat_list.append(df_part_normal)
    # df_part_normal=pd.DataFrame(part_normal,columns=list('ABCDEFGJKL'))
    # df_part_normal

    # 
    # 3.提取零件行-BSI配件
    part_BSI=re.findall(r'(8\S{8}6) (\S{11}) (\S+) (\S+) (\S+) (\S+) (\S+) (\S+)',i)
    part_BSI
    if part_BSI:
        df_part_BSI=pd.DataFrame(part_BSI,columns=list('ABCDFGKL'))
        concat_list.append(df_part_BSI)
        
     # 3.提取零件行-全陪付配件
    df_part_short_detail=re.findall(r'^(\S{10}) (\S{11}) (\S+) (\S+) (\S+) (\S+) (\S+) (\S+) (\S+)$',i)
    df_part_short_detail
    if df_part_short_detail:
        df_short=pd.DataFrame(df_part_short_detail,columns=list('ABCDFGJKL'))
        concat_list.append(df_short)
    # df_part_BSI=pd.DataFrame(part_BSI,columns=list('ABCDFGKL'))
    # df_part_BSI

    # 
    # 4.提取辅料
    sub_4=re.findall(r'(\S{10}) (\S{1}) (\S+) (\S+) (\S+) (\S+) (\S+) (\S+)',i)
    sub_4
    if sub_4:
        df_sub_4=pd.DataFrame(sub_4,columns=list('ABCDFIKL'))
        concat_list.append(df_sub_4)
        
    # df_sub_4=pd.DataFrame(sub_4,columns=list('ABCDFIKL'))
    # df_sub_4
    
    # 5.提取F111
    F111=re.findall(r'(9999001386) (4) (\S+) (\S+) (\S+) (\S+) (\S+)',detail_list[19])
    F111
    if F111:
        df_F111=pd.DataFrame(F111,columns=list('ABCFIKL'))
        concat_list.append(df_F111)
        
    # df_F111=pd.DataFrame(F111,columns=list('ABCFIKL'))
    # df_F111

    # 
    # 6.找到单行的表头
    p=r'(\d{5,6}) (\S{17}) (\S{6})'
    one_head=re.findall(p,i)
    one_head
    claim,vin,dwp=one_head[0]
    claim,vin,dwp

    # 
    # 建立一个空的dataframe，只包括表头-作用是避免漏掉没有数据的列
    df_na=pd.DataFrame(columns=list('ABCDEFGHIJKL'))
    concat_list.append(df_na)

    # 
    # 拼接dataframe
    res_one_part=pd.concat(concat_list,axis=0)
    # column 按照字母排序
    res_one_part.sort_index(axis=1,inplace=True)
    # claim vin dwp 插入前三列
    res_one_part.insert(0,'claim',claim)
    res_one_part.insert(1,'vin',vin)
    res_one_part.insert(2,'dwp',dwp)
    res_list.append(res_one_part)

res=pd.concat(res_list,axis=0)
# 将A-L列转换为数值
# res[list('ABCDEFJHIJKL')]=res[list('ABCDEFJHIJKL')].apply(pd.to_numeric,errors='ignore')
res

,claim,vin,dwp,A,B,C,D,E,F,G,H,I,J,K,L
0,212513,WMW51DK09N2R49823,X8CK7F,5122073700,5199000,31.90,100.0,NaN,4,NaN,127.60,NaN,NaN,16.59,144.19
0,212513,WMW51DK09N2R49823,X8CK7F,5122073700,51227497233,89.38,100.0,21.00-,1,70.61,NaN,NaN,5.08,9.84,85.53
0,212513,WMW51DK09N2R49823,X8CK7F,9999001386,4,713.00,NaN,NaN,1,NaN,NaN,713.00,NaN,92.69,805.69
0,212511,WBACY2105M9H82023,X8CK7G,0061740600,0072693,31.90,100.0,NaN,8,NaN,255.20,NaN,NaN,33.18,288.38
0,212511,WBACY2105M9H82023,X8CK7G,9999001386,4,713.00,NaN,NaN,1,NaN,NaN,713.00,NaN,92.69,805.69
0,212510,WMW41DL05N3N29495,X8CK7H,6411721200,0000556,31.90,100.0,NaN,2,NaN,63.80,NaN,NaN,8.29,72.09
1,212510,WMW41DL05N3N29495,X8CK7H,6411721200,6411841,31.90,100.0,NaN,4,NaN,127.60,NaN,NaN,16.59,144.19
0,212510,WMW41DL05N3N29495,X8CK7H,6411721200,64119384023,778.76,100.0,29.00-,1,552.92,NaN,NaN,39.81,77.05,669.78
0,212510,WMW41DL05N3N29495,X8CK7H,9999001386,4,713.00,NaN,NaN,1,NaN,NaN,713.00,NaN,92.69,805.69
0,212499,WMWXR3108M2P29472,X8CK7I,3234013200,3234030,31.90,100.0,NaN,4,NaN,127.60,NaN,NaN,16.59,144.19


In [193]:

# res.to_excel('res1.xlsx',index=False)